<a href="https://colab.research.google.com/github/daianekr/kaggle-challenge/blob/main/nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## House Prices Competions


**Introdução:** a competição do kaggle trata sobre o uso de variáveis explicativas que descreve vários aspectos de casas resisdenciais em Ames, Iowa, e que nos desafia a prever o preço final de cada casa.

O conjunto de dados nomeado de *Ames Housing Dataset* foi construído por Dean de Cock para o ensino de Ciência de Dados.

**Objetivo:** trabalhar para prver o preço de venda de cada casa. Para cada ID no conjunto de teste, devemos prever o valor da variável ***SalePrice*** do conjunto de teste.

**Métrica:** a avaliação dos envios é feita pela *RMSE (Root-Mean-Squared-Error)*, medindo o logaritmo do valor previsto e o logaritmo do preço de venda real da amostra. Fazer isso significa que erros na previsão de casas caras ou baratas efetarão o resultado igualmente.

**Formato do arquivo de submissão:** o arquivo deve conter um cabeçalho e ter o seguinte formato:

```
Id,SalePrice
1461,169000.1
1462,187724.1233
1463,175221
etc.

```



## Importando as Libs

In [11]:
import pandas as pd
#import kaggle as kg
#from scikit-learn import

### Lendo o arquivo de treino disponibilizado:


Visualizando as primeiras linhas do arquivo:


In [12]:
data = pd.read_csv('/train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Campos do conjunto de dados:


* **SalePrice**: preço de venda da propriedade em dólares. Esta é a variável alvo que você está tentando prever.
* **MSSubClass**: Classe do edifício.
* **MSZoning**: Classificação geral de zoneamento.
* **LotFrontage**: Metros lineares de rua conectados à propriedade.
* **LotArea**: Tamanho do lote em pés quadrados.
* **Street**: Tipo de acesso por estrada.
* **Alley**: Tipo de acesso por viela.
* **LotShape**: Formato geral da propriedade.
* **LandContour**: Nivelamento da propriedade.
* **Utilities**: Tipo de utilidades disponíveis.
* **LotConfig**: Configuração do lote.
* **LandSlope**: Inclinação da propriedade.
* **Neighborhood**: Localização física dentro dos limites da cidade de Ames.
* **Condition1**: Proximidade com estrada principal ou ferrovia.
* **Condition2**: Proximidade com estrada principal ou ferrovia (se houver uma segunda).
* **BldgType**: Tipo de habitação.
* **HouseStyle**: Estilo da habitação.
* **OverallQual**: Qualidade geral dos materiais e acabamentos.
* **OverallCond**: Avaliação da condição geral.
* **YearBuilt**: Data de construção original.
* **YearRemodAdd**: Data da reforma.
* **RoofStyle**: Tipo de telhado.
* **RoofMatl**: Material do telhado.
* **Exterior1st**: Revestimento exterior da casa.
* **Exterior2nd**: Revestimento exterior da casa (se houver mais de um material).
* **MasVnrType**: Tipo de revestimento em alvenaria.
* **MasVnrArea**: Área de revestimento em alvenaria em pés quadrados.
* **ExterQual**: Qualidade do material exterior.
* **ExterCond**: Condição atual do material exterior.
* **Foundation**: Tipo de fundação.
* **BsmtQual**: Altura do porão.
* **BsmtCond**: Condição geral do porão.
* **BsmtExposure**: Paredes do porão com saída para o exterior ou nível de jardim.
* **BsmtFinType1**: Qualidade da área acabada do porão.
* **BsmtFinSF1**: Área acabada do tipo 1 em pés quadrados.
* **BsmtFinType2**: Qualidade da segunda área acabada (se presente).
* **BsmtFinSF2**: Área acabada do tipo 2 em pés quadrados.
* **BsmtUnfSF**: Área inacabada do porão em pés quadrados.
* **TotalBsmtSF**: Área total do porão em pés quadrados.
* **Heating**: Tipo de aquecimento.
* **HeatingQC**: Qualidade e condição do aquecimento.
* **CentralAir**: Ar-condicionado central.
* **Electrical**: Sistema elétrico.
* **1stFlrSF**: Área do primeiro andar em pés quadrados.
* **2ndFlrSF**: Área do segundo andar em pés quadrados.
* **LowQualFinSF**: Área acabada de baixa qualidade (todos os andares) em pés quadrados.
* **GrLivArea**: Área útil acima do nível do solo em pés quadrados.
* **BsmtFullBath**: Banheiros completos no porão.
* **BsmtHalfBath**: Lavabos no porão.
* **FullBath**: Banheiros completos acima do nível do solo.
* **HalfBath**: Lavabos acima do nível do solo.
* **Bedroom**: Número de quartos acima do nível do porão.
* **Kitchen**: Número de cozinhas.
* **KitchenQual**: Qualidade da cozinha.
* **TotRmsAbvGrd**: Total de cômodos acima do nível do solo (não inclui banheiros).
* **Functional**: Avaliação da funcionalidade da casa.
* **Fireplaces**: Número de lareiras.
* **FireplaceQu**: Qualidade da lareira.
* **GarageType**: Localização da garagem.
* **GarageYrBlt**: Ano de construção da garagem.
* **GarageFinish**: Acabamento interior da garagem.
* **GarageCars**: Capacidade da garagem em número de carros.
* **GarageArea**: Tamanho da garagem em pés quadrados.
* **GarageQual**: Qualidade da garagem.
* **GarageCond**: Condição da garagem.
* **PavedDrive**: Entrada pavimentada para carros.
* **WoodDeckSF**: Área de deck de madeira em pés quadrados.
* **OpenPorchSF**: Área de varanda aberta em pés quadrados.
* **EnclosedPorch**: Área de varanda fechada em pés quadrados.
* **3SsnPorch**: Área de varanda para três estações em pés quadrados.
* **ScreenPorch**: Área de varanda com tela em pés quadrados.
* **PoolArea**: Área da piscina em pés quadrados.
* **PoolQC**: Qualidade da piscina.
* **Fence**: Qualidade da cerca.
* **MiscFeature**: Característica adicional não coberta em outras categorias.
* **MiscVal**: Valor em dólares da característica adicional.
* **MoSold**: Mês da venda.
* **YrSold**: Ano da venda.
* **SaleType**: Tipo de venda.
* **SaleCondition**: Condição da venda.



## Descrição estetística das variáveis numéricas:

In [16]:
for coluna in data.select_dtypes(include=['number']).columns:
    print(f"Coluna: {coluna}")
    print(data[coluna].describe())
    print("-" * 50)

Coluna: Id
count    1460.000000
mean      730.500000
std       421.610009
min         1.000000
25%       365.750000
50%       730.500000
75%      1095.250000
max      1460.000000
Name: Id, dtype: float64
--------------------------------------------------
Coluna: MSSubClass
count    1460.000000
mean       56.897260
std        42.300571
min        20.000000
25%        20.000000
50%        50.000000
75%        70.000000
max       190.000000
Name: MSSubClass, dtype: float64
--------------------------------------------------
Coluna: LotFrontage
count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64
--------------------------------------------------
Coluna: LotArea
count      1460.000000
mean      10516.828082
std        9981.264932
min        1300.000000
25%        7553.500000
50%        9478.500000
75%       11601.500000
max      215245.000000
N

## Tipagem dos dados:

In [13]:
for coluna, tipo in data.dtypes.items():
    print(f"{coluna}: {tipo}")

Id: int64
MSSubClass: int64
MSZoning: object
LotFrontage: float64
LotArea: int64
Street: object
Alley: object
LotShape: object
LandContour: object
Utilities: object
LotConfig: object
LandSlope: object
Neighborhood: object
Condition1: object
Condition2: object
BldgType: object
HouseStyle: object
OverallQual: int64
OverallCond: int64
YearBuilt: int64
YearRemodAdd: int64
RoofStyle: object
RoofMatl: object
Exterior1st: object
Exterior2nd: object
MasVnrType: object
MasVnrArea: float64
ExterQual: object
ExterCond: object
Foundation: object
BsmtQual: object
BsmtCond: object
BsmtExposure: object
BsmtFinType1: object
BsmtFinSF1: int64
BsmtFinType2: object
BsmtFinSF2: int64
BsmtUnfSF: int64
TotalBsmtSF: int64
Heating: object
HeatingQC: object
CentralAir: object
Electrical: object
1stFlrSF: int64
2ndFlrSF: int64
LowQualFinSF: int64
GrLivArea: int64
BsmtFullBath: int64
BsmtHalfBath: int64
FullBath: int64
HalfBath: int64
BedroomAbvGr: int64
KitchenAbvGr: int64
KitchenQual: object
TotRmsAbvGrd: int6

## Teste

In [ ]:
data = pd.read_csv('data/train.csv',index_col='Id')
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
data.shape

(1460, 80)

In [ ]:
data.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [ ]:
data.shape

(1460, 80)

In [ ]:
y = data.SalePrice


In [ ]:
data.drop(['SalePrice'], axis=1, inplace=True)